# PyCity Schools Analysis

Given the data, the district had a failing overall grade of 65.17% (this is with the normally grading metric of a 70% as passing). This data is misleading when divided into school type. It is my conclusion that the charter schools in the district are doing exceptionally well while the district schools are doing very poorly.

With an overall comparison of 90.43% passing rate for charter schools to a 53.67% passing rate for district schools, the district has a large bias towards charter schools. The top 5 best performing schools are charter schools while the bottom 5 schools are district schools. When looking into it further, the lowest overall average grade for charter schools is 89.23% while the highest district school overall grade was 54.64%. 

In attempting to analyze further differences between grade performance in schools, there is not enough data to determine anything beyond the difference of type. Student grade level does not seem to be a factor given that the grade level scores at each school is within +1 or -1 of the school average. Schools with smaller populations have better scores, but every charter school has fewer students than every district school. Furthermore, you might also conclude that the smaller budget per student means higher grades except for the fact that all but one of the charter schools has a smaller budget than district schools. Of the district schools that have a smaller budget than the one charter school, one has the highest overall grade and the other has the lowest. When looking at the school types individually, there is no obvious pattern between size of student population or budget per student compared to grades.

My conclusion is that a student in a charter school has a drastically better chance of passing math and reading than in the district schools and that there should be further research into the differences based on class size, course level, feeder middle schools, and student demographics to help better understand the differences in charter and district school performance.

In [68]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on="school_name")
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [69]:
# Calculate the total number of unique schools
school_count = school_data_complete.school_name.unique().size
school_count

15

In [70]:
# Calculate the total number of students
student_count = school_data_complete.student_name.count()
student_count

39170

In [71]:
# Calculate the total budget
total_budget = school_data.budget.sum()
total_budget

24649428

In [72]:
# Calculate the average (mean) math score
average_math_score = round(school_data_complete.math_score.mean(),2)
average_math_score

78.99

In [73]:
# Calculate the average (mean) reading score
average_reading_score = round(school_data_complete.reading_score.mean(),2)
average_reading_score

81.88

In [74]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = round(passing_math_count / float(student_count) * 100,2)
passing_math_percentage

74.98

In [75]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = round(passing_reading_count / float(student_count) * 100,2)
passing_reading_percentage

85.81

In [76]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = round(passing_math_reading_count /  float(student_count) * 100, 2)
overall_passing_rate

65.17

In [77]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame(
    {
        "Total Schools": [school_count],
    "Total Students" : [student_count],
    "Total Budget" : [total_budget],
    "Average Math Score" : [average_math_score],
    "Average Reading Score" : [average_reading_score],
    "% Passing Math" : [passing_math_percentage],
    "% Passing Reading" : [passing_reading_percentage],
    "% Overall Passing" : [overall_passing_rate]
    }
)

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary.reset_index()

,index,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

In [78]:
# Use the code provided to select all of the school types
school_types = school_data_complete.groupby("school_name")["type"].first()

In [79]:
# Calculate the total student count per school
per_school_counts = school_data_complete.groupby("school_name")["size"].first()

In [80]:
# Calculate the total school budget and per capita spending per school
per_school_budget = school_data_complete.groupby("school_name")["budget"].first()
#Column Metric for per student budget
per_student_budget = per_school_budget/per_school_counts

In [81]:
# Calculate the average test scores per school
per_school_math = round(school_data_complete.groupby("school_name")["math_score"].mean(),2)
per_school_reading = round(school_data_complete.groupby("school_name")["reading_score"].mean(),2)

In [82]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = school_data_complete[(school_data_complete["math_score"] >= 70)]
school_students_passing_math = students_passing_math.groupby(["school_name"]).size()

In [83]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)]
school_students_passing_reading = students_passing_reading.groupby(["school_name"]).size()

In [84]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

In [85]:
# Use the provided code to calculate the passing rates
per_school_passing_math = round(school_students_passing_math / per_school_counts * 100, 2)
per_school_passing_reading = round(school_students_passing_reading / per_school_counts * 100, 2)
overall_passing_rate = round(school_students_passing_math_and_reading / per_school_counts * 100, 2)

In [86]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
#Original DataFram is called school_summary to avoid formating mismatch in further use 
school_summary = pd.DataFrame(
    {
        "School Type" : school_types,
        "Total Students" : per_school_counts,
        "Total School Budget" : per_school_budget,
        "Per Student Budget" : per_student_budget,
        "Average Math Score" : per_school_math,
        "Average Reading Score" : per_school_reading,
        "% Passing Math" : per_school_passing_math,
        "% Passing Reading" : per_school_passing_reading,
        "% Overall Passing" : overall_passing_rate
    }
)

#Create DataFrame called per_school_summary to display information
per_school_summary = school_summary.copy()

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)
per_school_summary.index.name = "School Name"

# Display the DataFrame
per_school_summary.reset_index()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
3,Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
5,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23
7,Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


## Highest-Performing Schools (by % Overall Passing)

In [87]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = school_summary.sort_values("% Overall Passing", ascending=False)
top_schools.index.name = None
top_schools.head(5).reset_index()


,index,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,91.33
1,Thomas High School,Charter,1635,1043130,638.0,83.42,83.85,93.27,97.31,90.95
2,Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,90.60
3,Wilson High School,Charter,2283,1319574,578.0,83.27,83.99,93.87,96.54,90.58
4,Pena High School,Charter,962,585858,609.0,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

In [88]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = school_summary.sort_values("% Overall Passing", ascending=True)
bottom_schools.index.name = None
bottom_schools.head(5).reset_index()

,index,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Rodriguez High School,District,3999,2547363,637.0,76.84,80.74,66.37,80.22,52.99
1,Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,53.20
2,Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,53.51
3,Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,53.53
4,Johnson High School,District,4761,3094650,650.0,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

In [89]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grader_math_scores = round(ninth_graders.groupby("school_name")["math_score"].mean(), 2)
tenth_grader_math_scores = round(tenth_graders.groupby("school_name")["math_score"].mean(), 2)
eleventh_grader_math_scores = round(eleventh_graders.groupby("school_name")["math_score"].mean(), 2)
twelfth_grader_math_scores = round(twelfth_graders.groupby("school_name")["math_score"].mean(), 2)

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame(
    {
        "9th" : ninth_grader_math_scores,
        "10th" : tenth_grader_math_scores,
        "11th" : eleventh_grader_math_scores,
        "12th" : twelfth_grader_math_scores,
        "Average" : per_school_math
    }
)

# Minor data wrangling
math_scores_by_grade.index.name = "School Name"

# Display the DataFrame
math_scores_by_grade.reset_index()

,School Name,9th,10th,11th,12th,Average
0,Bailey High School,77.08,77.00,77.52,76.49,77.05
1,Cabrera High School,83.09,83.15,82.77,83.28,83.06
2,Figueroa High School,76.40,76.54,76.88,77.15,76.71
3,Ford High School,77.36,77.67,76.92,76.18,77.10
4,Griffin High School,82.04,84.23,83.84,83.36,83.35
5,Hernandez High School,77.44,77.34,77.14,77.19,77.29
6,Holden High School,83.79,83.43,85.00,82.86,83.80
7,Huang High School,77.03,75.91,76.45,77.23,76.63
8,Johnson High School,77.19,76.69,77.49,76.86,77.07
9,Pena High School,83.63,83.37,84.33,84.12,83.84


## Reading Score by Grade 

In [107]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grader_reading_scores = round(ninth_graders.groupby("school_name")["reading_score"].mean(), 2)
tenth_grader_reading_scores = round(tenth_graders.groupby("school_name")["reading_score"].mean(), 2)
eleventh_grader_reading_scores = round(eleventh_graders.groupby("school_name")["reading_score"].mean(), 2)
twelfth_grader_reading_scores = round(twelfth_graders.groupby("school_name")["reading_score"].mean(), 2)

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame( 
    {
        "9th" : ninth_grader_reading_scores,
        "10th" : tenth_grader_reading_scores,
        "11th" : eleventh_grader_reading_scores,
        "12th" : twelfth_grader_reading_scores,
        "Average" : per_school_reading
      }
)

# Minor data wrangling
reading_scores_by_grade.index.name = "School Name"

# Display the DataFrame
reading_scores_by_grade.reset_index()

,School Name,School Type,9th,10th,11th,12th,Average
0,Bailey High School,District,81.30,80.91,80.95,80.91,81.03
1,Cabrera High School,Charter,83.68,84.25,83.79,84.29,83.98
2,Figueroa High School,District,81.20,81.41,80.64,81.38,81.16
3,Ford High School,District,80.63,81.26,80.40,80.66,80.75
4,Griffin High School,Charter,83.37,83.71,84.29,84.01,83.82
5,Hernandez High School,District,80.87,80.66,81.40,80.86,80.93
6,Holden High School,Charter,83.68,83.32,83.82,84.70,83.81
7,Huang High School,District,81.29,81.51,81.42,80.31,81.18
8,Johnson High School,District,81.26,80.77,80.62,81.23,80.97
9,Pena High School,Charter,83.81,83.61,84.34,84.59,84.04


## Scores by School Spending

In [91]:
# Establish the bins 
bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [92]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = school_summary.copy()

In [93]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], bins, labels)

# Formatting
school_spending_df["Total School Budget"] = school_spending_df["Total School Budget"].map("${:,.2f}".format)
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].map("${:,.2f}".format)
school_spending_df.index.name = "School Name"

school_spending_df.reset_index()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64,"(585, 630]"
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33,"(0, 585]"
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20,"(630, 645]"
3,Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29,"(630, 645]"
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60,"(585, 630]"
5,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53,"(645, 680]"
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23,"(0, 585]"
7,Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51,"(645, 680]"
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54,"(645, 680]"
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54,"(585, 630]"


In [94]:
#  Calculate averages for the desired columns. 
spending_math_scores = round(school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean(),2)
spending_reading_scores = round(school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean(),2)
spending_passing_math = round(school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean(),2)
spending_passing_reading = round(school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean(),2)
overall_passing_spending = round(school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean(),2)

In [95]:
# Assemble into DataFrame
spending_summary = pd.DataFrame(
    {
       "Average Math Score" : spending_math_scores,
        "Average Reading Score" : spending_reading_scores,
        "% Passing Math" : spending_passing_math,
        "% Passing Reading" : spending_passing_reading,
        "% Overall Passing" : overall_passing_spending
    }
)

# Display results
spending_summary.reset_index()

,Spending Ranges (Per Student),Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,"(0, 585]",83.45,83.94,93.46,96.61,90.37
1,"(585, 630]",81.90,83.16,87.13,92.72,81.42
2,"(630, 645]",78.52,81.62,73.48,84.39,62.86
3,"(645, 680]",77.00,81.03,66.16,81.13,53.53


## Scores by School Size

In [96]:
# Establish the bins.
bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [97]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `school_summary` DataFrame.
school_size_summary = school_summary.copy()
school_size_summary["School Size"] =  pd.cut(school_size_summary["Total Students"], bins, labels)
school_size_summary.index.name = "School Name"
school_size_summary.reset_index()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
0,Bailey High School,District,4976,3124928,628.0,77.05,81.03,66.68,81.93,54.64,"(2000, 5000]"
1,Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,91.33,"(1000, 2000]"
2,Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,53.20,"(2000, 5000]"
3,Ford High School,District,2739,1763916,644.0,77.10,80.75,68.31,79.30,54.29,"(2000, 5000]"
4,Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,90.60,"(1000, 2000]"
5,Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,53.53,"(2000, 5000]"
6,Holden High School,Charter,427,248087,581.0,83.80,83.81,92.51,96.25,89.23,"(0, 1000]"
7,Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,53.51,"(2000, 5000]"
8,Johnson High School,District,4761,3094650,650.0,77.07,80.97,66.06,81.22,53.54,"(2000, 5000]"
9,Pena High School,Charter,962,585858,609.0,83.84,84.04,94.59,95.95,90.54,"(0, 1000]"


In [98]:
# Calculate averages for the desired columns. 
size_math_scores = round(school_size_summary.groupby(["School Size"])["Average Math Score"].mean(), 2)
size_reading_scores = round(school_size_summary.groupby(["School Size"])["Average Reading Score"].mean(), 2)
size_passing_math = round(school_size_summary.groupby(["School Size"])["% Passing Math"].mean(), 2)
size_passing_reading = round(school_size_summary.groupby(["School Size"])["% Passing Reading"].mean(), 2)
size_overall_passing = round(school_size_summary.groupby(["School Size"])["% Overall Passing"].mean(), 2)

In [99]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame(
    {
       "Average Math Score" : size_math_scores,
        "Average Reading Score" : size_reading_scores,
        "% Passing Math" : size_passing_math,
        "% Passing Reading" : size_passing_reading,
        "% Overall Passing" : size_overall_passing
    }
)

# Display results
size_summary.reset_index()

,School Size,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,"(0, 1000]",83.82,83.93,93.55,96.10,89.88
1,"(1000, 2000]",83.37,83.87,93.60,96.79,90.62
2,"(2000, 5000]",77.74,81.34,69.96,82.77,58.28


## Scores by School Type

In [100]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = round(school_summary.groupby(["School Type"])["Average Math Score"].mean(), 2)
average_reading_score_by_type = round(school_summary.groupby(["School Type"])["Average Reading Score"].mean(), 2)
average_percent_passing_math_by_type = round(school_summary.groupby(["School Type"])["% Passing Math"].mean(), 2)
average_percent_passing_reading_by_type = round(school_summary.groupby(["School Type"])["% Passing Reading"].mean(), 2)
average_percent_overall_passing_by_type = round(school_summary.groupby(["School Type"])["% Overall Passing"].mean(), 2)

In [101]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame(
    {
       "Average Math Score" : average_math_score_by_type,
        "Average Reading Score" : average_reading_score_by_type,
        "% Passing Math" : average_percent_passing_math_by_type,
        "% Passing Reading" : average_percent_passing_reading_by_type,
        "% Overall Passing" : average_percent_overall_passing_by_type
    }
)

# Display results
type_summary.reset_index()

,School Type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Charter,83.47,83.90,93.62,96.59,90.43
1,District,76.96,80.97,66.55,80.80,53.67


# EXTRA Code for Analysis

In [106]:
#sort code by spending ranges per student
spending_by_school_type = school_spending_df.sort_values("Per Student Budget", ascending=True)
spending_by_school_type

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
School Name,,,,,,,,,,
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,90.58,"(0, 585]"
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23,"(0, 585]"
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33,"(0, 585]"
Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.68,83.96,93.33,96.61,90.33,"(0, 585]"
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.36,83.73,93.87,95.85,89.89,"(585, 630]"
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54,"(585, 630]"
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60,"(585, 630]"
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64,"(585, 630]"
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,52.99,"(630, 645]"


In [105]:
#sort code by spending ranges per student
size_by_school_type = school_size_summary.sort_values("Total Students", ascending=True)
size_by_school_type

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
School Name,,,,,,,,,,
Holden High School,Charter,427,248087,581.0,83.80,83.81,92.51,96.25,89.23,"(0, 1000]"
Pena High School,Charter,962,585858,609.0,83.84,84.04,94.59,95.95,90.54,"(0, 1000]"
Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,90.60,"(1000, 2000]"
Thomas High School,Charter,1635,1043130,638.0,83.42,83.85,93.27,97.31,90.95,"(1000, 2000]"
Shelton High School,Charter,1761,1056600,600.0,83.36,83.73,93.87,95.85,89.89,"(1000, 2000]"
Wright High School,Charter,1800,1049400,583.0,83.68,83.96,93.33,96.61,90.33,"(1000, 2000]"
Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,91.33,"(1000, 2000]"
Wilson High School,Charter,2283,1319574,578.0,83.27,83.99,93.87,96.54,90.58,"(2000, 5000]"
Ford High School,District,2739,1763916,644.0,77.10,80.75,68.31,79.30,54.29,"(2000, 5000]"
